In [47]:
import evaluate
import process
text = 'data/brown.txt'

In [21]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import casual

import json

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import numpy as np
import argparse
import numpy as np
from scipy.stats import spearmanr

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cosine
from numpy.linalg import norm

import random
import os

from process import load_model, load_msr

In [13]:
windowList = [2,5,10]
dimList = [100,300,1000]
EPOCHS = 5
iwin = 2
idim = 100

In [44]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        for line in open(self.filename, 'r'):
            yield [word.lower() for word in tokenizer.tokenize(line) if word.lower() not in stop_words]
sentence = MySentences(text)

In [49]:
# count unigrams
from collections import Counter
word_counts = Counter()
for x in sentence:
    word_counts.update(x)

In [56]:
print(len(word_counts))
print(word_counts.most_common(5))
print(word_counts.most_common()[-5:])

42281
[('one', 3504), ('would', 2719), ('said', 1961), ('time', 1695), ('new', 1646)]
[('perelman', 1), ('exhaling', 1), ('aviary', 1), ('boucle', 1), ('stupefying', 1)]


In [55]:
# minimum word count = 5
word_counts2 = Counter()
for key,value in word_counts.items():
    if value >= 5:
        word_counts2[key] = value
print(len(word_counts2))
print(word_counts2.most_common(5))
print(word_counts2.most_common()[-5:])

13891
[('one', 3504), ('would', 2719), ('said', 1961), ('time', 1695), ('new', 1646)]
[('furrow', 5), ('richert', 5), ('kafka', 5), ('poitrine', 5), ('quasimodo', 5)]


In [66]:
# count co-occurance
joint = Counter()
for isen in sentence:
    for j, word in enumerate(isen):
        if word not in word_counts2:
            continue
        index_min = max(0, j-iwin)
        index_max = min(len(isen), j+iwin+1)
        index = [ii for ii in range(index_min, index_max) if ii!=j]
        for iin in index:
            if isen[iin] in word_counts2:
                joint[(word, isen[iin])] += 1
print(len(joint))
print(joint.most_common(5))

1197681
[(('united', 'states'), 401), (('states', 'united'), 401), (('af', 'af'), 392), (('new', 'york'), 312), (('york', 'new'), 312)]


In [82]:
word_counts2['new']

1646

In [69]:
# generate index for words in vocabulary
index_dict = dict()
ii = 0
for i in word_counts2:
    index_dict[i] = ii
    ii += 1
    
import math
row_index = []
col_index = []
values = []
for (wi,wj), count in joint.items():
    row = index_dict[wi]
    col = index_dict[wj]
    prod = word_counts2[wi]*word_counts2[wj]
    if count > prod:
        value = math.log(count/prod)
    else:
        value = 0
    row_index.append(row)
    col_index.append(col)
    values.append(value)

from scipy import sparse
ppmi = sparse.csr_matrix((values, (row_index, col_index)))

In [72]:
ppmi.getrow(0)

<1x13891 sparse matrix of type '<class 'numpy.int64'>'
	with 48 stored elements in Compressed Sparse Row format>

In [42]:
text = "Sentences iterable can be simply a list, but for larger corpora, consider a generator that streams the sentences directly from disk/network"

In [79]:
print(ppmi[1,:])

  (0, 0)	0
  (0, 1)	0
  (0, 2)	0
  (0, 3)	0
  (0, 4)	0
  (0, 5)	0
  (0, 7)	0
  (0, 9)	0
  (0, 17)	0
  (0, 18)	0
  (0, 19)	0
  (0, 20)	0
  (0, 30)	0
  (0, 31)	0
  (0, 32)	0
  (0, 34)	0
  (0, 48)	0
  (0, 67)	0
  (0, 68)	0
  (0, 69)	0
  (0, 76)	0
  (0, 81)	0
  (0, 106)	0
  (0, 109)	0
  (0, 117)	0
  :	:
  (0, 7701)	0
  (0, 7787)	0
  (0, 7788)	0
  (0, 7880)	0
  (0, 8104)	0
  (0, 8212)	0
  (0, 8417)	0
  (0, 8496)	0
  (0, 8713)	0
  (0, 8757)	0
  (0, 9131)	0
  (0, 9355)	0
  (0, 9395)	0
  (0, 10491)	0
  (0, 10527)	0
  (0, 10607)	0
  (0, 10871)	0
  (0, 11048)	0
  (0, 11158)	0
  (0, 11431)	0
  (0, 11490)	0
  (0, 12293)	0
  (0, 12342)	0
  (0, 12866)	0
  (0, 13685)	0


In [80]:
ppmi.nonzero()

(array([], dtype=int32), array([], dtype=int32))